<a href="https://colab.research.google.com/github/MahanNasirkhani/Mask-Detection/blob/main/MobileSAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

HOME: /content


In [ ]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
import cv2
import mediapipe as mp
import itertools
from tqdm import tqdm

 # Install MobileSAM

In [ ]:
%cd {HOME}

!pip install git+https://github.com/ChaoningZhang/MobileSAM.git
!wget https://raw.githubusercontent.com/ChaoningZhang/MobileSAM/master/weights/mobile_sam.pt

/content
  Cloning https://github.com/ChaoningZhang/MobileSAM.git to /tmp/pip-req-build-v307648w
  Running command git clone --filter=blob:none --quiet https://github.com/ChaoningZhang/MobileSAM.git /tmp/pip-req-build-v307648w
  Resolved https://github.com/ChaoningZhang/MobileSAM.git to commit 01ea8d0f5590082f0c1ceb0a3e2272593f20154b
  Preparing metadata (setup.py) ... done
--2023-08-20 08:15:55--  https://raw.githubusercontent.com/ChaoningZhang/MobileSAM/master/weights/mobile_sam.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40728226 (39M) [application/octet-stream]
Saving to: ‘mobile_sam.pt.1’

mobile_sam.pt.1     100%[===================>]  38.84M  --.-KB/s    in 0.1s    

2023-08-20 08:15:56 (348 MB/s) - ‘mobile_sam.pt.1’ saved [40728226/40728226]



# Load MobileSAM

In [ ]:
from mobile_sam import sam_model_registry, SamPredictor

sam_checkpoint = f"{HOME}/mobile_sam.pt"
model_type = "vit_t"
device = "cuda" if torch.cuda.is_available() else "cpu"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
sam.eval()

predictor = SamPredictor(sam)

# Load Dataset

In [ ]:
base_dir = '/content/drive/MyDrive/Dataset'
train_dir = os.path.join(base_dir, 'Train')
test_dir = os.path.join(base_dir, 'Test')

In [ ]:
def load_images_from_folder(folder):
    images = []
    labels = []
    class_names = os.listdir(folder)

    for class_name in class_names:
        class_folder = os.path.join(folder, class_name)
        if not os.path.isdir(class_folder):
            continue

        for filename in tqdm(os.listdir(class_folder)):
            img_path = os.path.join(class_folder, filename)
            img = cv2.imread(img_path)
            image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            if img is not None:
                images.append(image)
                labels.append(class_name)

    return np.array(images), np.array(labels)

In [ ]:
X_train_data, y_train_data = load_images_from_folder(train_dir)
X_test_data, y_test_data = load_images_from_folder(test_dir)

100%|██████████| 560/560 [00:13<00:00, 43.06it/s]
<ipython-input-12-88a220b4d37c>:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(images), np.array(labels)


# Removing Darkness to Reveal Image Features

In [ ]:
def delete_dark(extracted_image, face_points):

  original_h = extracted_image.shape[0]
  original_w = extracted_image.shape[1]

  i = 0
  x_1 = 0
  y_1 = 0
  x_2 = original_h -1
  y_2 = original_w -1

  while True:
    if (extracted_image[0][i].all()!=0):
      y_1 = i
      break
    if (extracted_image[original_h//4][i].all()!=0):
      y_1 = i
      break
    if (extracted_image[original_h//2][i].all()!=0):
      y_1 = i
      break
    if (extracted_image[3*(original_h//4)][i].all()!=0):
      y_1 = i
      break
    if (extracted_image[original_h-1][i].all()!=0):
      y_1 = i
      break
    i = i + original_w//20
    if (i>=original_w):
      break

  i = 0
  while True:
    if (extracted_image[i][0].all()!=0):
      x_1 = i
      break
    if (extracted_image[i][original_w//4].all()!=0):
      x_1 = i
      break
    if (extracted_image[i][original_w//2].all()!=0):
      x_1 = i
      break
    if (extracted_image[i][3*(original_w//4)].all()!=0):
      x_1 = i
      break
    if (extracted_image[i][original_w-1].all()!=0):
      x_1 = i
      break
    i = i + original_h//20
    if (i>=original_h):
      break


  i = original_w -1
  while True:
    if (extracted_image[0][i].all()!=0):
      y_2 = i
      break
    if (extracted_image[original_h//4][i].all()!=0):
      y_2 = i
      break
    if (extracted_image[original_h//2][i].all()!=0):
      y_2 = i
      break
    if (extracted_image[3*(original_h//4)][i].all()!=0):
      y_2 = i
      break
    if (extracted_image[original_h-1][i].all()!=0):
      y_2 = i
      break
    i = i - original_w//20
    if (i<0):
      break

  i = original_h -1
  while True:
    if (extracted_image[i][0].all()!=0):
      x_2 = i
      break
    if (extracted_image[i][original_w//4].all()!=0):
      x_2 = i
      break
    if (extracted_image[i][original_w//2].all()!=0):
      x_2 = i
      break
    if (extracted_image[i][3*(original_w//4)].all()!=0):
      x_2 = i
      break
    if (extracted_image[i][original_w-1].all()!=0):
      x_2 = i
      break
    i = i - original_h//20
    if (i<0):
      break

  if (x_1 == x_2) or (y_1 == y_2) or (x_1 > x_2) or (y_1 > y_2):
    y_1 = face_points[0][0]
    y_2 = face_points[2][0]
    x_1 = face_points[3][1]
    x_2 = face_points[1][1]


  return extracted_image[x_1:x_2, y_1:y_2]

# Loading face_mesh and Defining Its Points

In [ ]:
points = [212,200,432,0]

In [ ]:
mp_face_mesh = mp.solutions.face_mesh

face_mesh_images = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=2)

# Facial Landmark Detection: Utilizing face_mesh to Identify Facial Points

In [ ]:
def face_points(image):

        for _ in range(5):
              face_mesh_results = face_mesh_images.process(image)

        FACEMESH_FACE_INDEXES_total = list(set(itertools.chain(*mp_face_mesh.FACEMESH_TESSELATION)))
        list_of_points_face = []
        if face_mesh_results.multi_face_landmarks:

                for face_landmarks in face_mesh_results.multi_face_landmarks:


                    for FACEMESH_FACE_INDEX in FACEMESH_FACE_INDEXES_total:

                        land = face_landmarks.landmark[FACEMESH_FACE_INDEX]
                        list_of_points_face.append((land.x,land.y))



                list_of_points_face = np.array(list_of_points_face)
                return list_of_points_face
        return "No Face Was Detected"

# Isolating and Highlighting the Face in Images

In [ ]:
def keep_face(image,list_of_points_face,original_w,original_h):
  list_of_points_face = list_of_points_face *[original_w , original_h]
  list_of_points_face = np.round(list_of_points_face)
  list_of_points_face = np.array(list_of_points_face ,np.int32)

  convexhull = cv2.convexHull(list_of_points_face)
  mask = np.zeros(image.shape[0:2], np.uint8)
  cv2.fillConvexPoly(mask, convexhull, 255)
  face = cv2.bitwise_and(image, image, mask=mask)

  return face

# Using MobileSAM for segmentation

In [ ]:
def segment(image,label):
  segment_part = []
  labels = []
  for i in tqdm(range(len(image))):
        original_h = image[i].shape[0]
        original_w = image[i].shape[1]

        list_of_points_face = face_points(image[i])

        input_points = []
        if type(list_of_points_face) != str:
                for j in points:
                  input_points.append((list_of_points_face[j][0],list_of_points_face[j][1]))

                input_points = np.array(input_points)
                input_points = input_points*[original_w , original_h]
                input_points = np.round(input_points)
                input_points = np.array(input_points, dtype=np.int16)
                input_label = np.ones((len(input_points)),dtype=np.int16)

                face = keep_face(image[i],list_of_points_face,original_w,original_h)

                predictor.set_image(face)
                masks, scores, logits = predictor.predict(
                       point_coords=input_points,
                      point_labels=input_label,
                     multimask_output=True,
                                    )

                max_score = masks[int(np.argmax(scores))]
                extracted_image = face * max_score[:, :, np.newaxis]

                result = delete_dark(extracted_image,input_points)

                segment_part.append(result)
                labels.append(label[i])

  return segment_part,labels

In [ ]:
X_train_keep,y_train_keep = segment(X_train_data,y_train_data)
X_test_keep,y_test_keep = segment(X_test_data,y_test_data)

100%|██████████| 1120/1120 [06:48<00:00,  2.74it/s]


# Saving Processed Data: Preserving Enhanced Images Generated by the Code

In [ ]:
train_dir_mask = '/content/drive/MyDrive/Mask Dataset/Train/Mask'
train_dir_non_mask = '/content/drive/MyDrive/Mask Dataset/Train/No Mask'
os.makedirs(train_dir_mask, exist_ok=True)
os.makedirs(train_dir_non_mask, exist_ok=True)

for idx, (image, label) in tqdm(enumerate(zip(X_train_keep, y_train_keep))):
    if label == "Mask":
      image_path = os.path.join(train_dir_mask, f'image_{idx}.jpg')
    else:
      image_path = os.path.join(train_dir_non_mask, f'image_{idx}.jpg')
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    cv2.imwrite(image_path, image_rgb)

In [ ]:
test_dir_mask = '/content/drive/MyDrive/Mask Dataset/Test/Mask'
test_dir_non_mask = '/content/drive/MyDrive/Mask Dataset/Test/No Mask'
os.makedirs(test_dir_mask, exist_ok=True)
os.makedirs(test_dir_non_mask, exist_ok=True)

for idx, (image, label) in tqdm(enumerate(zip(X_test_keep, y_test_keep))):
    if label == "Mask":
      image_path = os.path.join(test_dir_mask, f'image_{idx}.jpg')
    else:
      image_path = os.path.join(test_dir_non_mask, f'image_{idx}.jpg')
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    cv2.imwrite(image_path, image_rgb)

1026it [05:32,  3.09it/s]
